In [1]:
import tensorflow as tf
import numpy as np
import cv2
import random
import glob
import copy

In [2]:
# Opening both face and non-faced images 
face_images = glob.glob("data/faces/**.jpg")
non_face_images = glob.glob("data/non-faces/**.jpg")
print(len(face_images), len(non_face_images))


450 1370


In [3]:
read_face_images = []
read_non_face_images = []

for i in range(len(face_images)):
    single_face_image = cv2.imread(face_images[i])
    single_face_image = cv2.resize(single_face_image, (224, 224))
    read_face_images.append(single_face_image)
    

for i in range(len(non_face_images)):
    non_single_face_image = cv2.imread(non_face_images[i])
    non_single_face_image = cv2.resize(non_single_face_image, (224, 224))
    read_non_face_images.append(non_single_face_image)
    

In [4]:
image_shape = read_non_face_images[0].shape
print(image_shape)


(224, 224, 3)


In [5]:
# Create clone of image and rotate for more data
def image_rotation(img_list):
    for i in range(len(img_list)):
        rand_number = random.randint(45, 180) 
        img = img_list[i]
        cloned_image = img.copy()
        M = cv2.getRotationMatrix2D((112,112), rand_number, 1)
        dst = cv2.warpAffine(img, M, (112,112))
        img_list.append(img)

In [6]:
import skimage

def add_noise(img_list):
    for i in range(len(img_list)):
        img = img_list[i]
        cloned_image = img.copy()
        noisy_image = skimage.util.random_noise(cloned_image, mode='gaussian', seed=None, clip=True) 
        img_list.append(np.asarray(noisy_image, dtype="uint8"))

In [9]:
from scipy.ndimage.interpolation import rotate
def sharpen_img(img_list):
    for i in range(len(img_list)):
        img = img_list[i]
        cloned_image = img.copy()
        gb = cv2.GaussianBlur(cloned_image, (5,5), 20.0)
        img_list.append(cv2.addWeighted(img, 2, gb, -1, 0))
        

In [10]:
def augment_all(img_list):
    image_rotation(img_list)
    add_noise(img_list)
    sharpen_img(img_list)
    
augment_all(read_face_images)
augment_all(read_non_face_images)

print(len(read_face_images), len(read_non_face_images))
    
    
    

3600 10960


In [11]:
# Concactenate the Images together
features_data = np.vstack((read_face_images, read_non_face_images)).astype(np.float64)

In [14]:
#Creating labels for the images
labels = np.hstack((np.ones(len(read_face_images)), np.zeros(len(read_non_face_images))))
print(len(features_data))

14560


In [19]:
from sklearn.cross_validation import train_test_split
random.shuffle(features_data)
X_train, X_test, y_train, y_test = train_test_split(features_data, labels, test_size=0, random_state = np.random.randint(0, 100))


In [17]:
#Defining Architecture

EPOCHS = 40
BATCH_SIZE = 120


In [18]:
#Architecture of the Convolutional Neural Network Begins Here
#Architecture comes from MobileNet Paper

from tensorflow.contrib.layers import flatten

def mobileNet(x):
    # Hyperparameters
    mu = 0
    sigma = 0.1
    stride = [1, 1, 1, 1]
    stride_2 = [1, 2, 2, 1]
    dropout = 0.5
    
    #Layer Input = 224, 224, 3 Output = 
    weights_0 = tf.Variable(tf.truncated_normal(shape= (3, 3, 3, 32), mean= mu, stddev= sigma))
    bias_0 = tf.Variable(tf.zeros(32))
    
    conv_0 = tf.nn.conv2d(x, weights_0, stride_2, padding= 'VALID') + bias_0
    
    #Activation
    conv_0 = tf.nn.relu(conv_0)
